In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

%cd /content/drive/My\ Drive/
%cd broutanlab_course/

Mounted at /content/drive
/content/drive/My Drive
/content/drive/My Drive/broutanlab_course


### Face Recognition

In [4]:
!pip install face_recognition

     |████████████████████████████████| 100.2MB 73kB/s 
  Created wheel for face-recognition-models: filename=face_recognition_models-0.3.0-py2.py3-none-any.whl size=100566173 sha256=f6b46dbd129866dda9e486646866fc0d3bdce4e8f340f668b238fed5c0f43871
  Stored in directory: /root/.cache/pip/wheels/d2/99/18/59c6c8f01e39810415c0e63f5bede7d83dfb0ffc039865465f
Successfully built face-recognition-models


In [5]:
import dlib
import face_recognition
from sklearn import svm
import os
import cv2
from google.colab.patches import cv2_imshow


In [7]:
# Training the SVC classifier

# The training data would be all the face encodings from all the known images and the labels are their names
encodings = []
names = []

# Training directory
train_dir = os.listdir('dataset3/')

# Loop through each person in the training directory
for person in train_dir:
    pix = os.listdir("dataset3/" + person)

    # Loop through each training image for the current person
    for person_img in pix:
        # Get the face encodings for the face in each image file
        face = face_recognition.load_image_file("dataset3/" + person + "/" + person_img)
        face_bounding_boxes = face_recognition.face_locations(face)

        #If training image contains exactly one face
        if len(face_bounding_boxes) == 1:
            face_enc = face_recognition.face_encodings(face)[0]
            # Add face encoding for current image with corresponding label (name) to the training data
            encodings.append(face_enc)
            names.append(person)
        else:
            print(person + "/" + person_img + " was skipped and can't be used for training")



Unknown/Ryan-Gosling.jpg was skipped and can't be used for training


In [8]:
# Create and train the SVC classifier
clf = svm.SVC(gamma='scale')
clf.fit(encodings,names)



SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [10]:
# Load the test image with unknown faces into a numpy array
from imutils import paths
imagePaths = list(paths.list_images('test3'))
for (i, imagePath) in enumerate(imagePaths):
  test_image = face_recognition.load_image_file(imagePath)

  # Find all the faces in the test image using the default HOG-based model
  face_locations = face_recognition.face_locations(test_image)
  no = len(face_locations)
  print("Number of faces detected: ", no)

  # Predict all the faces in the test image using the trained classifier
  cv2_imshow(test_image)
  print("Found:")
  for i in range(no):
      test_image_enc = face_recognition.face_encodings(test_image)[i]
      name = clf.predict([test_image_enc])
      print(*name)

Output hidden; open in https://colab.research.google.com to view.